In [8]:
import os
import sys

sys.path.append(os.path.abspath("../../"))

from typing import Type

import matplotlib.pyplot as plt
import numpy as np
import torch
from transformers import AutoConfig, AutoProcessor, AutoTokenizer

from utils.collator import SequenceGenerationCollator
from utils.dataset import ERCDataset, IemocapDataset, MeldDataset
from utils.model import MmLlamaConcat, MmLlamaConfig
from utils.processor import MmLlamaProcessor

# Early concatenation

In diesem Abschnitt werden die erzielten Ergebnisse und das Early-Concatenation Modell weitergehend untersucht, um folgenden Forschungsfragen zu beantworten:
- Wie ist die performance auf IEMOCAP und MELD
- Gibt es Änderungen in der Klassifizierung im Vergleich zum normalen InstructERC
  - Wenn ja, welche?
- Was wird durch die Akustik erkannt?
  - Was wird nur durch Akustik erkannt?
  - Gibt es Verbesserungen in bestimmten Emotionen
- Kann das Modell das volle Potenzial aus beiden Modalitäten ausnutzen?
- Nutzt das Modell die neuen Feature?
- Hat das Vortraining einen Einfluss auf die Effektivität des Modells

## Attention-Weights

Um zu untersuchen, ob das training effektiv und das Modell die zusätzlichen feature tatsächlich verwendet, untersuchen wir die Attention-Weight Matrix.
Diese gibt aufschluss darüber, worauf sich das Modell konzentriert.
Zur erinnerung, die Attention-Matrix ist eine $T_x \times T_x$ Matrix, wobei $T_x$ für die Länge der Eingabesequenz steht.
Diese ist besitzt nur im unteren linken Dreieck $\{A_{i,j} \mid i \ge j\}$ Werte, während alle anderen 0 betragen.
Eine Begründung dafür steht in Abschnitt (ref...).
Jede Zeile enthält das Attenntion-Query Ergebnis eines Query-Vektors, während die Spalten die Ergebnisse der Keys representiert.
Zu interpretieren ist also eine Zeile, dass sich aus der neue Kontext-Vektor für den Token zu prozentualen Anteilen zusammensetzt, wie diese in der Zeile stehen.
Währenddessen zeigt jede Zeile, wie stark ein gegebener Token zum nächsten Zustand beigetragen hat.
Befinden sich in einer Spalte nur niedrige Werte, hat dieser Token wenig zum Ergebnisausgang beigetragen.

Wenn man also überprüfen möchte, ob das Netz die akustischen Feature tatsächlich verwendet, sollte es keine besonderen Auffälligkeiten oder erhöhte Werte in den Spalten der Audio-Token geben.

In [9]:
LANGUAGE_MODEL = os.path.abspath("../../models/language/LLaMA2-base")
LORA_ADAPTER = os.path.abspath("../../models/language/adapter/iemocap/LLaMA2-base")
ACOUSTIC_MODEL = os.path.abspath(
    "../../models/acoustic/wav2vec2/wav2vec2-large-robust-12-ft-emotion-msp-dim"
)
DS_TRAIN_PATH = os.path.abspath("../../datasets/iemocap/iemocap.csv")
DS_DEV_PATH = os.path.abspath("../../datasets/iemocap/iemocap.csv")
DS_TEST_PATH = os.path.abspath("../../datasets/iemocap/iemocap.csv")
STAGE1_PATH = os.path.abspath(
    "../../experiments/multimodal/concat/iemocap/LLaMA2-base/mlp/audio_instruction/stage_1"
)
STAGE2_PATH = os.path.abspath(
    "../../experiments/multimodal/concat/iemocap/LLaMA2-base/mlp/audio_instruction/stage_1"
)

In [12]:
model = None
config = None
processor = None


def get_model(
    llm_path: str, adapter_path: str, acoustic_path: str, checkpoint_path: str
):
    global model, config, processor
    if None in (model, config, processor):
        return model, config, processor

    llm_config = AutoConfig.from_pretrained(llm_path)
    ac_config = AutoConfig.from_pretrained(acoustic_path)
    ac_processor = AutoProcessor.from_pretrained(acoustic_path)

    # setup of tokenizer
    tokenizer = AutoTokenizer.from_pretrained(llm_path)
    tokenizer.add_special_tokens({"additional_special_tokens": ["<audio>"]})
    tokenizer.pad_token_id = tokenizer.unk_token_id
    tokenizer.padding_side = "left"

    # setup of processor
    processor = MmLlamaProcessor(ac_processor, tokenizer)

    ## setup of config
    audio_token_id = tokenizer.additional_special_tokens_ids[0]
    config = MmLlamaConfig(
        llm_config=llm_config,
        audio_config=ac_config,
        audio_token_id=audio_token_id,
        pad_token_id=tokenizer.pad_token_id,
        llm_pretrained_adapter=adapter_path,
        num_labels=0,
    )

    model = MmLlamaConcat(config, output_attention_weights=True)
    model.load_state_dict(
        torch.load(os.path.join(checkpoint_path, "best_model.pth")), strict=False
    )
    model = model.apply_inference_lora(checkpoint_path)
    if torch.cuda.is_available():
        model = model.to("cuda")

    return model, config, processor


def get_sample(dataset_path: str, sample_index=0):
    def dataset_class(dataset_path: str) -> Type[ERCDataset]:
        if "meld" in dataset_path:
            return MeldDataset
        if "iemocap" in dataset_path:
            return IemocapDataset
        else:
            raise ValueError("Invalid dataset path")

    test_dataset = dataset_class(dataset_path)(
        dataset_path, mode="test", task="normal", audio_placement="target"
    )
    raw_sample = test_dataset[sample_index]
    sample = SequenceGenerationCollator(processor, mode="dev")([raw_sample])
    sample = sample[0]

    return sample


def get_attention_weights(
    model: MmLlamaConcat, dataset_path: str, layer_idx=0, sample_index=0
):
    sample = get_sample(dataset_path, sample_index)

    llama = model.llama
    att1 = llama.get_submodule(f"model.layers.{layer_idx}.self_attn")

    attention_weights = None

    def attention_hook(module, input, output):
        nonlocal attention_weights
        attention_weights = output[1]

    att_handle = att1.register_forward_hook(attention_hook)

    def prepate_nested_batch(batch: dict[dict[torch.Tensor]]):
        device = "cpu"
        if torch.cuda.is_available():
            device = "cuda"

        text = {k: v.to(device) for k, v in batch["text"].items()}
        acoustic = {k: v.half().to(device) for k, v in batch["acoustic"].items()}

        return {**text, **acoustic}

    with torch.no_grad():
        _ = model(**prepate_nested_batch(sample))

    att_handle.remove()
    return attention_weights[0].cpu().numpy()


def print_attention_weight_matrix(
    attention_weights: np.ndarray,
    sample,
    config: MmLlamaConfig,
    tokenizer: AutoTokenizer,
):
    head_norm = attention_weights.mean(axis=0)
    head_norm = np.apply_along_axis(lambda x: x / np.max(x), 1, head_norm)

    audio_token_id = config.audio_token_id

    token_ids = sample["text"]["input_ids"][0].cpu().numpy()
    audio_loc = np.where(token_ids == audio_token_id)[0][0]
    token_ids = np.concatenate(
        (token_ids[:audio_loc], [audio_token_id] * 10, token_ids[audio_loc:])
    )
    token_strings = np.array(tokenizer.convert_ids_to_tokens(token_ids))
    # token_strings[token_ids == 0] = ""

    mean_attention = np.mean(head_norm, axis=0, where=head_norm != 0)
    # print(mean_attention)
    top_token_ids_idx = np.argsort(mean_attention)[::-1][:25]
    # print(top_token_ids_idx)
    # last_token_ids_idx = np.argsort(mean_attention)[:10]
    audio_loc_idx = np.where(token_ids == audio_token_id)[0]
    selected_tokens = np.unique(np.concatenate([top_token_ids_idx, audio_loc_idx]))
    token_strings[~np.isin(np.arange(len(token_strings)), selected_tokens)] = ""
    # print(token_strings)

    plt.figure(figsize=(10, 10))
    plt.imshow(head_norm, cmap="viridis")
    plt.xticks(range(len(token_strings)), token_strings, rotation=90, fontsize=6)
    plt.yticks(range(len(token_strings)), token_strings, fontsize=6)
    plt.xlabel("Query Tokens")
    plt.ylabel("Key Tokens")
    plt.show()


In [11]:
def print_model_attention_weights(dataset_path: str, layer_idx=0, sample_index=0):
    model, config, processor = get_model(
        LANGUAGE_MODEL, LORA_ADAPTER, ACOUSTIC_MODEL, STAGE1_PATH
    )
    attention_weights = get_attention_weights(
        model, dataset_path, layer_idx, sample_index
    )
    print_attention_weight_matrix(
        attention_weights, sample_index, config, processor.tokenizer
    )


print_model_attention_weights(layer_idx=0, sample_index=0)

Um herauszufinden, welchen Einfluss das Vortraining auf die Verteilung der GEwichte hat, ist hier die Matrix eines Modells, welches direkt auf Stage 3 trainiert wurde.